# Introduction to Gradient Boosting

Gradient boosting is the idea of using multiple models in an ensemble and focusing on the residuals in the machine learning process. Using the same HMEQ Data, we will be able to show that focusing on residuals through a second model can be effective to lowering the error on a test set.

## Import Relevant Libraries and Data

In [1]:
import numpy as np
import pandas as pd
from sklearn import tree

from sklearn.model_selection import train_test_split
from pprint import pprint

In [2]:
filename = 'https://github.com/Humboldt-WI/bads/blob/master/data/hmeq_modeling.csv?raw=true'
df = pd.read_csv(filename, header = 0, index_col = 0)

In [3]:
X = df.drop(['BAD'], axis=1) #code the variables in the most standard way for your usage
y = df[['BAD']]

X.head() #inspect that variables were correctly separated

,LOAN,MORTDUE,VALUE,YOJ,CLAGE,NINQ,CLNO,DEBTINC,DEROGzero,REASON_HomeImp,REASON_IsMissing,JOB_Office,JOB_Other,JOB_ProfExe,JOB_Sales,JOB_Self,DELINQcat_1,DELINQcat_1+
index,,,,,,,,,,,,,,,,,,
0,-1.832283,-1.295882,-1.335526,0.266788,-1.075278,-0.065054,-1.297476,0.137456,True,1,0,0,1,0,0,0,0,0
1,-1.810666,-0.013474,-0.672699,-0.236615,-0.723092,-0.826792,-0.756608,0.137456,True,1,0,0,1,0,0,0,0,1
2,-1.789048,-1.654549,-1.839275,-0.668103,-0.368769,-0.065054,-1.189302,0.137456,True,1,0,0,1,0,0,0,0,0
3,-1.789048,-0.159552,-0.202559,-0.236615,-0.061033,-0.065054,-0.107566,0.137456,True,0,1,0,1,0,0,0,0,0
4,-1.767431,0.791699,0.311107,-0.811933,-1.088528,-0.826792,-0.756608,0.137456,True,1,0,1,0,0,0,0,0,0


In [4]:
y.head()

,BAD
index,
0,True
1,True
2,True
3,True
4,False


In [5]:
# train test split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=123)

In [6]:
print(type(X_train), type(y_train)) # double check that types and dimensions are correct before proceeding

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [7]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(4768, 18) (4768, 1) (1192, 18) (1192, 1)


## Gradient Boosting from Scratch Example: Training Models

Here we will show the effectiveness of corrective models which work on the principle of boosting: training on errors. We will first train two models, the first will be for regular predictions. The second will predict which observations may lead to errors. We will first run the first prediction on test data, then correct these predictions using the second model.


In [8]:
estimators = []

In [9]:
clf = tree.DecisionTreeClassifier(criterion="entropy", min_samples_split=2, max_depth=2) #first classifier

dt = clf.fit(X_train, y_train) #fit the classifier

estimators.append(('first model', dt))

In [10]:
initial_pred = dt.predict(X_train) #predict using first classifier

In [11]:
res = initial_pred != y_train.iloc[:,0] #check residuals
res.mean()

0.15121644295302014

In [12]:
res.sum() #total errors of this classifier

721

In [13]:
clf2 = tree.DecisionTreeClassifier(criterion="gini") #train second classifier with different specs

dt_res = clf2.fit(X_train, res) #fit classifier on the residuals, since we are training on decisions of a binary outcome, this classifier will predict errors of the first classifier
estimators.append(('second model', dt_res))
dt_res

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [14]:
likely_misclassifications = dt_res.predict(X_train) #the results of the first classifier are errors of the first classifier
likely_misclassifications.sum()

721

In [37]:
accuracy_misclassifications = likely_misclassifications == res #check if classified likely misclassifications are the same as residuals
accuracy_misclassifications

index
706     True
3647    True
4807    True
1250    True
3748    True
        ... 
5218    True
4060    True
1346    True
3454    True
3582    True
Name: BAD, Length: 4768, dtype: bool

In [39]:
accuracy_misclassifications.mean() #it seems likely misclassifications are exactly congruent with residuals, so the model does work

1.0

## Gradient Boosting from Scratch Example: Testing Models

Now that we have our two models, we will begin using the test data to see if it is able to bring down the value of the residuals. We will first predict y using X_test.

In [19]:
pred_initial_test = dt.predict(X_test)

In [20]:
res_test = pred_initial_test != y_test.iloc[:,0]
res_test.mean()

0.14093959731543623

Now we predict for which observations the classifier would likely have gotten the predictions incorrect.

In [21]:
likely_misclassifications_test = dt_res.predict(X_test)
likely_misclassifications_test

array([False, False, False, ...,  True, False, False])

Lastly, we correct the likely misclassifications by classifying them the opposite way.

In [22]:
pred_corrected = pd.Series(pred_initial_test)
pred_corrected[likely_misclassifications_test] = ~ pred_corrected[likely_misclassifications_test]

In [23]:
pred_initial_test[likely_misclassifications_test] #check that they have actually been changed

array([False,  True, False,  True, False, False,  True,  True, False,
        True,  True, False, False, False, False,  True, False,  True,
        True,  True, False, False,  True, False,  True,  True, False,
        True, False,  True, False,  True,  True,  True,  True, False,
       False, False, False,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True, False, False,
       False,  True,  True, False, False, False,  True,  True,  True,
        True, False, False,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True, False,  True, False,
        True,  True,  True,  True, False, False, False, False,  True,
        True,  True, False,  True, False,  True,  True,  True,  True,
       False, False,  True, False,  True,  True, False, False, False,
       False, False,  True,  True,  True, False, False, False,  True,
        True,  True,  True,  True,  True, False,  True,  True, False,
        True,  True,

In [24]:
pred_corrected[likely_misclassifications_test] #all the results are opposite, so this worked!

5        True
13      False
17       True
20      False
26       True
        ...  
1165     True
1175    False
1176     True
1182    False
1189    False
Length: 159, dtype: object

In [25]:
res_corrected = np.array(pred_corrected) != y_test.iloc[:,0]
res_corrected.mean()

0.13003355704697986

## Conclusion

The new residuals are smaller than the previous ones. This tells us that our process worked. We were able to lower the error on a test set using a second model which only focused on identifying residuals.

Gradient boost is able to do this process in multiple ways with much more complex methods and algorithms. However, in essence, the idea is that multiple models are trained and some specifically target residuals. AdaBoost and XGBoost are both popular algorithms which focus on this principle.